In [1]:
import json
import os
from chromadb.config import Settings
from langchain_community.chat_models import ChatOllama
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [2]:
# Initialize embeddings, vector store, and model
embeddings = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-embedding",
    model_kwargs={"device": "cuda"},
)

setting = Settings(
    anonymized_telemetry=False,
    is_persistent=True,
)

vector_store = Chroma(
    persist_directory='../../.DB_llama3',
    embedding_function=embeddings,
    client_settings=setting,
)

retriever = vector_store.as_retriever()

model = ChatOllama(model="llama3")

/tmp/ipykernel_1638/1959779872.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/hungcq/study/AI_Chatbot/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
docs = retriever.invoke("Đà Nẵng")
docs

[Document(metadata={}, page_content='- Theo ông, đâu là những yếu tố trọng yếu giúp Đà Nẵng là điểm đến thu hút du khách\ntrong và ngoài nước suốt thời gian qua? - Những năm qua, Đà Nẵng không chỉ đóng góp về lượng khách, còn trở thành một\ntrong những thương hiệu du lịch gắn với Việt Nam, cả về tài nguyên, hạ tầng, hệ thống\ndịch vụ, công tác phục vụ và môi trường điểm đến. Có 3 yếu tố quan trọng giúp Đà Nẵng đạt được thành công này.'),
 Document(metadata={}, page_content="'Du lịch Đà Nẵng theo hướng bền vững sẽ không lãng phí tài\nnguyên'\nTrên cơ sở tài nguyên, hạ tầng sẵn có, Đà Nẵng cần sự đồng hành của các nhà đầu tư\nđủ tầm để phát triển theo hướng bền vững, theo ông Cao Trí Dũng, Chủ tịch Hiệp hội Du\nlịch Đà Nẵng. Đà Nẵng đang được quy hoạch để có những khu du lịch tầm cỡ thế giới. 15 năm qua,\nthành phố có bước nhảy vọt về phát triển du lịch, từ chỗ nghèo nàn thiếu thốn hạ tầng,\ntrở thành một trong những điểm đến hút khách bậc nhất cả nước. Trong kế hoạch phát\ntriển du lịch

In [4]:
docs = retriever.invoke("Tại sao không được đi chân trần trên máy bay?")
docs

[Document(metadata={}, page_content='Thế nhưng, hãng hàng không Qantas của Úc thậm chí còn không cho phép khách đi\ndép trong phòng chờ, theo One Mile At A Time. Thật không may, việc bay lên trời mà không đi tất không chỉ bị coi là thô lỗ mà còn tiềm\nẩn nguy cơ gây hại cho sức khỏe, bởi sàn máy bay rất bẩn. "Tôi sẽ không bao giờ đi chân trần trên máy bay", một tiếp viên hàng không kỳ cựu của\nhãng hàng không lớn cho biết, theo WGN Morning News.'),
 Document(metadata={}, page_content='Lý do các hãng hàng không Mỹ cấm hành khách đi chân trần trên\nmáy bay\nHành khách đi chân trần trên máy bay ngày càng đông nhưng không phải tất cả các\nhãng hàng không đều chấp nhận, nhiều hãng Mỹ cấm bay nếu khách không đi tất. Để lộ chân trần trên máy bay có thể khiến bạn bị "đá" khỏi một số hãng hàng không Mỹ. Định nghĩa "chân trần" bao gồm cả việc cởi tất khi đã lên máy bay.'),
 Document(metadata={}, page_content='Các chuyên gia về chuyến bay trên diễn đàn hàng không One Mile At A Time lập luận\nquy 

In [5]:
def format_markdown_line(string: str) -> str:
    return f"\n{string}\n".replace(
        "\n",
        """
""",
    )


def process_questions_from_json(json_path: str, output_md_file: str):
    try:
        # Step 1: Load questions from JSON file
        with open(json_path, "r") as f:
            questions_list = json.load(f)

        # Step 2: Initialize a variable to store previous question's result
        previous_response_content = ""

        # Step 3: Loop through each question in the list
        for index, question_data in enumerate(questions_list):
            question = question_data.get("question")
            is_next_question = question_data.get("is_next_question")
            ref = question_data.get("ref")
            truth = question_data.get("truth")

            # Step 4: Retrieve the documents
            invoking_question = (
                f"{question} \nPrevious answer: {previous_response_content}"
                if is_next_question is True
                else question
            )
            retrieved_docs = retriever.invoke(invoking_question)
            context = "\n".join([doc.page_content for doc in retrieved_docs])

            # If there's a previous question, add its result to the context
            if is_next_question is True:
                context += format_markdown_line(
                    f"Câu trả lời trước đó: {previous_response_content}"
                )

            # Step 5: Generate the final prompt
            prompt_template = PromptTemplate.from_template(
                """
                <s> [INST] Bạn là trợ lý hữu ích, hãy trả lời các câu hỏi về các tài liệu tin tức đã nhập. 
                Chỉ sử dụng ngữ cảnh được cung cấp, không sử dụng bất kỳ thông tin nào ngoài ngữ cảnh này. 
                Nếu bạn không biết hoặc ngữ cảnh không liên quan tới câu hỏi, chỉ cần nói rằng bạn không biết. 
                Bạn chỉ được trả lời bằng tiếng Việt. 
                [/INST] </s> 
                [INST] Câu hỏi: {question} 
                Ngữ cảnh: {context} 
                Câu trả lời: [/INST]
                """
            )
            prompt = prompt_template.format(question=question, context=context)

            # Step 6: Send the prompt to the model
            response = model.invoke(prompt)

            # Extract model's response content and retrieved document content
            model_response_content = response.content
            retrieved_docs_content = "".join(
                [
                    format_markdown_line(f"* {doc.page_content}")
                    for doc in retrieved_docs
                ]
            )

            # Step 7: Append results to markdown file
            try:
                with open(output_md_file, "a") as md_file:
                    md_file.write(
                        format_markdown_line(f"## Câu hỏi #{index+1}: {question}")
                    )
                    if ref:
                        md_file.write(
                            format_markdown_line(f"**Tham khảo:**\n\n{ref}")
                        )
                    if truth:
                        md_file.write(
                            format_markdown_line(f"**Sự thật:**\n\n{truth}")
                        )
                    md_file.write(
                        format_markdown_line(
                            f"**Câu trả lời:**\n\n{model_response_content}"
                        )
                    )
                    if is_next_question is True:
                        md_file.write(
                            format_markdown_line(
                                f"**Câu trả lời trước đó:**\n\n{previous_response_content}"
                            )
                        )
                    md_file.write(
                        format_markdown_line(
                            f"**Tài liệu được lấy ra:**\n\n{retrieved_docs_content}"
                        )
                    )
            except Exception as e:
                print(f"Error appending to markdown file: {e}")
                continue

            # Step 8: Update previous response content for the next question
            previous_response_content = model_response_content

    except Exception as e:
        print(f"Error processing questions: {e}")

In [6]:
# Example usage
json_path = 'medical_questions.json'
output_md_file = 'medical_results.md'
process_questions_from_json(json_path, output_md_file)